In [1]:
import numpy as np
import pandas as pd
import altair as alt
import bokeh.io
import bokeh.layouts
import bokeh.palettes
import bokeh.plotting
from bokeh.models import Legend
from bokeh.models import Range1d
# import bebi103
# import altair_catplot as alt_cat
import re
import scipy.stats as st
import glob
import os

bokeh.io.output_notebook()

palette = bokeh.palettes.Colorblind6


Loading BokehJS ...

### Inoculation times were 13.51 ks, 85.75 ks, and 68.6 ks, respectively, for 2019_08_28, 2019_10_07, and 2019_10_20

In [2]:
def current_plotter(df, palette, title='Current during electrolysis', show_legend=True, width=800, height=400, spike_times=np.array([])):
    
    current_fig = bokeh.plotting.figure(height=height,
                                width=width,
                                y_axis_type='log',
                                y_axis_label='Current (µA)',
                                x_axis_label='Time (days)',
                                title=title)

    grouped = df.groupby(['exp', 'chamber'])

    legend_items = []

    for i, g in enumerate(grouped):
        exp = g[0][0]
        ch = g[0][1]
        gdf = g[1]

        l = current_fig.line(gdf['time (days)'], gdf['current (uA)'], color=palette[int(ch)], line_width=3, muted_color=palette[int(ch)], muted_alpha=0.1)
        
        if len(spike_times) > 0:
            spike_times = list(spike_times)
            ts = [(t * 1000 / (60 * 60 * 24), t * 1000 / (60 * 60 * 24)) for t in spike_times]
            ys = [(3 * 10 ** 2, 5 * 10 ** 2) for t in spike_times]
            
            mlt = current_fig.multi_line(ts, ys, color='black', line_width=3)
        
        legend_items.append((f'Chamber {int(ch)} {exp}', [l]))
    
    if show_legend:
        legend = Legend(items=legend_items)
        
        current_fig.add_layout(legend, 'right')
        current_fig.legend.click_policy = 'mute'
        
    else:
        pass
    

    
    return current_fig

In [3]:
def get_spiking_potentiostat_df(date, inoculation_ks, path='../../../data/EChem'):
    
    dfs = []
    
    for p in ['p0', 'p1', 'p2', 'p3']:
        path_to_data = os.path.join(path, f'{date}_EChemSurvival_Spiking_{p}.txt')
        df = pd.read_csv(path_to_data, delimiter='\t')
        df['chamber'] = p[1]
        df['exp'] = date
        
        dfs.append(df)
    
    poten_df = pd.concat(dfs)
    poten_df = poten_df[['exp', 'chamber', 'time (s)', 'charge (C)', 'current (A)']]
    
    poten_df['time (s)'] = poten_df['time (s)'] - inoculation_ks * 1000
    
    poten_df['time (days)'] = poten_df['time (s)'] / (60 * 60 * 24)
    poten_df['current (uA)'] = poten_df['current (A)'] * 1e6

        
        
    return poten_df

In [4]:
spiking_C = pd.read_csv('../data/echem/2019_11_03_EChemSurvival_Spiking_p2.txt', delimiter='\t')
spiking_C['chamber'] = '2'
spiking_C['exp'] = '2019_11_04'

spiking_C = spiking_C[['exp', 'chamber', 'time (s)', 'charge (C)', 'current (A)']]

spiking_C['time (days)'] = spiking_C['time (s)'] / (60 * 60 * 24)
spiking_C['current (uA)'] = spiking_C['current (A)'] * 1e6

In [5]:
spiking_C

,exp,chamber,time (s),charge (C),current (A),time (days),current (uA)
0,2019_11_04,2,0.0,0.000053,0.000011,0.000000,10.520
1,2019_11_04,2,5.0,0.000080,0.000005,0.000058,5.423
2,2019_11_04,2,10.0,-0.004680,0.000952,0.000116,951.900
3,2019_11_04,2,15.0,-0.006632,0.000390,0.000174,390.400
4,2019_11_04,2,20.0,-0.008107,0.000295,0.000231,295.100
...,...,...,...,...,...,...,...
84593,2019_11_04,2,423000.0,-39.530000,0.000132,4.895833,132.400
84594,2019_11_04,2,423000.0,-39.530000,0.000132,4.895833,132.000
84595,2019_11_04,2,423000.0,-39.530000,0.000133,4.895833,132.900
84596,2019_11_04,2,423000.0,-39.530000,0.000133,4.895833,132.600


In [6]:
spike_times = np.array([143, 186.2, 229.4, 272.6, 315.8, 359.1]) - 99.8
spiking_C_plot = current_plotter(spiking_C[::200], palette)
bokeh.io.show(spiking_C_plot)

In [7]:
spiking_df = get_spiking_potentiostat_df('2019_11_03', 99.8, path='../data/echem/')
spiking_df.head()

,exp,chamber,time (s),charge (C),current (A),time (days),current (uA)
0,2019_11_03,0,-99800.0,0.000072,0.000014,-1.155093,14.480
1,2019_11_03,0,-99795.0,0.000109,0.000007,-1.155035,7.384
2,2019_11_03,0,-99790.0,-0.003387,0.000699,-1.154977,699.200
3,2019_11_03,0,-99785.0,-0.005144,0.000351,-1.154919,351.400
4,2019_11_03,0,-99780.0,-0.006405,0.000252,-1.154861,252.300


In [8]:
spiking_df_from_zero = spiking_df.loc[spiking_df['time (days)'] > 0]
spiking_plot = current_plotter(spiking_df_from_zero[::200], palette, title='Spiking', spike_times=spike_times)

In [9]:
bokeh.io.show(spiking_plot)

In [10]:
spike_times[::2]

array([ 43.2, 129.6, 216. ])

In [11]:
spiking_plots = []

chamber_dict = {
    0: '0 mM nitrate',
    1: '5 mM nitrate',
    2: '5 mM nitrate, respike every 24 hours',
    3: '5 mM nitrate, respike every 12 hours'
}

for ch in [0, 1, 2, 3]:
    df = spiking_df_from_zero.loc[spiking_df_from_zero['chamber'] == str(ch)]
    if ch not in [2, 3]:
        p = current_plotter(df[::200], palette, title = chamber_dict[ch], show_legend=False, width=400, height=200)
    elif ch == 2:
        p = current_plotter(df[::200], palette, title = chamber_dict[ch], show_legend=False, width=400, height=200, spike_times=spike_times[1::2])
    elif ch == 3:
        p = current_plotter(df[::200], palette, title = chamber_dict[ch], show_legend=False, width=400, height=200, spike_times=spike_times)
    
    p.xaxis.major_label_text_font_size = '14pt'
    p.yaxis.major_label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_size = '14pt'
    p.yaxis.axis_label_text_font_size = '14pt'
    p.title.text_font_size = '14pt'
    p.output_backend = 'svg'
    
    spiking_plots.append(p)

for p in spiking_plots[:-1]:
    p.y_range = spiking_plots[-1].y_range
    p.x_range = spiking_plots[-1].x_range
    
    
spiking_faceted = bokeh.layouts.gridplot(spiking_plots, ncols=2)    
bokeh.io.show(spiking_faceted)

In [12]:
bokeh.io.export_svg(spiking_faceted, filename='./plots/figure17.svg')

['./plots/figure17.svg']